In [33]:
for (a, b) in enumerate('abcv'):
    print (a,b)

(0, 'a')
(1, 'b')
(2, 'c')
(3, 'v')


In [1]:
import numpy as np

data = open('one_digit_plus.txt').read()
chars = list(set(data))
nc = len(chars)
char2idx = {ch:idx for idx, ch in enumerate(chars)}
idx2char = {idx:ch for idx, ch in enumerate(chars)}

# params
r = 0.01
sigma = 0.01
hn = 50
ufdl = 20
Wgx = np.random.randn(hn, nc) * sigma
Wgh = np.random.randn(hn, hn) * sigma
Bg = np.zeros([hn, 1])
Wix = np.random.randn(hn, nc) * sigma
Wih = np.random.randn(hn, hn) * sigma
Bi = np.zeros([hn, 1])
Wfx = np.random.randn(hn, nc) * sigma
Wfh = np.random.randn(hn, hn) * sigma
Bf = np.zeros([hn, 1])
Wox = np.random.randn(hn, nc) * sigma
Woh = np.random.randn(hn, hn) * sigma
Bo = np.zeros([hn, 1])
Wkh = np.random.randn(nc, hn) * sigma
Bk = np.zeros([nc, 1])

sigmoid = lambda x:  1.0 /(1.0 + np.exp(-x))
softmax = lambda x: np.exp(x) / np.sum( np.exp(x) )

def update(xdata, zdata, prevs, prevh):
    dWgx, dWgh, dWix, dWih = np.zeros_like(Wgx), np.zeros_like(Wgh), np.zeros_like(Wix), np.zeros_like(Wih)
    dWfx, dWfh, dWox, dWoh, dWkh = np.zeros_like(Wfx), np.zeros_like(Wfh), np.zeros_like(Wox), np.zeros_like(Woh), np.zeros_like(Wkh)
    dBg, dBi, dBf, dBo, dBk = np.zeros_like(Bg), np.zeros_like(Bi), np.zeros_like(Bf), np.zeros_like(Bo), np.zeros_like(Bk)
    
    # forward pass
    loss = 0
    Ykt, Ht, Gt, It, Ft, Ot, St = {},{},{},{},{},{},{}
    for t in range(ufdl):
        Xt = np.zeros([nc, 1])
        Xt[ char2idx[ xdata[t]] ] = 1
        Zt = np.zeros([nc, 1])
        Zt[ char2idx[ zdata[t]] ] = 1
        
        Gt[t] = sigmoid(Wgx.dot(Xt) + Wgh.dot(prevh) + Bg)
        It[t] = sigmoid(Wix.dot(Xt) + Wih.dot(prevh) + Bi)
        Ft[t] = sigmoid(Wfx.dot(Xt) + Wfh.dot(prevh) + Bf)
        Ot[t] = sigmoid(Wox.dot(Xt) + Woh.dot(prevh) + Bo) 
        
        St[t] = Gt[t] * It[t] + prevs * Ft[t]
        prevs = St[t]
       # Ht[t] = sigmoid(St[t]) * Ot[t]
        Ht[t] = np.tanh(St[t]) * Ot[t]
        prevh = Ht[t]
        Akt = Wkh.dot(Ht[t]) + Bk
        Ykt[t] = softmax(Akt)
        loss = loss - np.sum(Zt * np.log(Ykt[t]))
        
    # backward pass
    dGr, dIr, dFr, dOr, dSp1 = np.zeros_like(Gt[0]), np.zeros_like(It[0]), np.zeros_like(Ft[0]), np.zeros_like(Ot[0]), np.zeros_like(St[0])
    Gt[ufdl], It[ufdl], Ft[ufdl], Ot[ufdl] = np.zeros_like(Gt[0]), np.zeros_like(It[0]), np.zeros_like(Ft[0]), np.zeros_like(Ot[0])
    St[-1] = np.zeros_like(St[0])
    Ht[-1] = np.zeros_like(Ht[0])
    for t in np.arange(ufdl, 0, -1)-1:
        Xt = np.zeros([nc, 1])
        Xt[ char2idx[ xdata[t]] ] = 1
        Zt = np.zeros([nc, 1])
        Zt[ char2idx[ zdata[t]] ] = 1
        dKt = Ykt[t] - Zt
        dWkh += dKt.dot(np.transpose(Ht[t]))
        dBk += dKt
        dHt = ( np.transpose(Wkh).dot(dKt) + (np.transpose(Wgh).dot(dGr) + (np.transpose(Wih).dot(dIr)) +
                                             (np.transpose(Wfh).dot(dFr)) + (np.transpose(Woh).dot(dOr)) ))
        
      #  dOt = dHt * sigmoid(St[t])
        dOt = dHt * np.tanh(St[t])
      #  dSt = dHt * Ot[t] * sigmoid(St[t])*(1-sigmoid(St[t])) + dSp1 * Ft[t+1]
        dSt = dHt * Ot[t] * (1-np.tanh(St[t])**2 + dSp1 + Ft[t+1])
        dGt = dSt * It[t]
        dIt = dSt * Gt[t]
        dFt = dSt * St[t-1]
        
        dGr = dGt * sigmoid(Wox.dot(Xt) + Woh.dot(Ht[t-1]) + Bo) * (1 - sigmoid(Wox.dot(Xt) + Woh.dot(Ht[t-1]) + Bo) ) 
        dWox += dGr.dot( np.transpose(Xt) )
        dWoh += dGr.dot( np.transpose(Ht[t-1]))
        dBo += dGr
        
        dFr = dFt * sigmoid(Wfx.dot(Xt) + Wfh.dot(Ht[t-1]) + Bf) * (1 - sigmoid(Wfx.dot(Xt) + Wfh.dot(Ht[t-1]) + Bf) )
        dWfx += dFr * np.transpose(Xt)
        dWfh += dFr * np.transpose(Ht[t-1])
        dBf += dFr
        
        dIr = dIt * sigmoid(Wix.dot(Xt) + Wih.dot(Ht[t-1]) + Bi) * (1 - sigmoid(Wix.dot(Xt) + Wih.dot(Ht[t-1]) + Bi) )
        dWix += dIr * np.transpose(Xt)
        dWih += dIr * np.transpose(Ht[t-1])
        dBi += dIr
        
        dGr = dGt * sigmoid(Wgx.dot(Xt) + Wgh.dot(Ht[t-1]) + Bg) * (1 - sigmoid(Wgx.dot(Xt) + Wgh.dot(Ht[t-1]) + Bg) )
        dWgx += dGr * np.transpose(Xt)
        dWgh += dGr * np.transpose(Ht[t-1])
        dBg += dGr
        
        dSp1 = dSt
    return St[ufdl-1], Ht[ufdl-1], loss, dWkh, dBk, dWox, dWoh, dBo, dWfx, dWfh, dBf, dWix, dWih, dBi, dWgx, dWgh, dBg
        
def sample(sc, steps, prevs, prevh): 
    Xt = np.zeros([nc, 1])
    Xt[ char2idx[ sc ] ] = 1
    output = sc
    
    for t in range(steps):
        Gt = sigmoid(Wgx.dot(Xt) + Wgh.dot(prevh) + Bg)
        It = sigmoid(Wix.dot(Xt) + Wih.dot(prevh) + Bi)
        Ft = sigmoid(Wfx.dot(Xt) + Wfh.dot(prevh) + Bf)
        Ot = sigmoid(Wox.dot(Xt) + Woh.dot(prevh) + Bo) 
        
        St = Gt * It + prevs * Ft
  #      print np.linalg.norm(St)
        prevs = St
        Ht = sigmoid(St) * Ot
        prevh = Ht
        Akt = Wkh.dot(Ht) + Bk
        Ykt = softmax(Akt)
        
        idx = np.random.choice(nc, p = Ykt.reshape([len(Ykt)]))
        output += idx2char[idx]
        Xt = np.zeros_like(Xt)
        Xt[idx] = 1
    print output
    
tlength = len(data)
for p in range(30):
    L, pt = 0, 0
    prevs, prevh = np.zeros([hn, 1]), np.zeros([hn, 1])
    for j in range(tlength / ufdl - 1):
        prevs, prevh, loss, dWkh, dBk, dWox, dWoh, dBo, dWfx, dWfh, dBf, dWix, dWih, dBi, dWgx, dWgh, dBg = (
            update(data[pt: pt+ufdl], data[pt+1: pt+ufdl+1], prevs, prevh )  )
        
        Wkh -= r*dWkh 
        Bk  -= r*dBk
        Wox -= r*dWox
        Woh -= r*dWoh
        Bo  -= r*dBo
        Wfx -= r*dWfx
        Wfh -= r*dWfh
        Bf  -= r*dBf
        Wix -= r*dWix
        Wih -= r*dWih
        Bi  -= r*dBi
        Wgx -= r*dWgx
        Wgh -= r*dWgh
        Bg  -= r*dBg
        L += loss
        
    print L
    
    sample('5', 200, np.zeros([hn, 1]), np.zeros([hn, 1]) )
        
        

57745.4398971
5+
3+=++8+3+
+3+=+4+
36
2+=
=+
==++
3+1=+++0
3+4=
=6=+=+=87==
+=37
42=3
+++4
+++01+6+=0+01+6+=3+2

++++31=++++
+3+6+6+23++
++8++32=1=
=6+2+
3+
63++6+
+++=8+6++
1=3

6++
7=

6

+1+=86683=
=++=36
+3+7+0=
23332.8717501
5+7=6
0
6+4=6
=
3+++74
4++4+4=6
3+4=3
6+0
6+7=12+8=

0
3
7
2+0=6+7=
3+70
3
2+8=1
70=00
6+8=13

6+7=
30
3+8=13+7=0=
0+8=13+8=6

++4
6+6+40=2+8+0
6++=13
0
3
2++7=1

+3
6+8=13
+3
6
6+8=12+4=6
3
2+8=3
6+4=
9716.89511396
5++7=
0+4=+7=6
3++8=
3
3++=13
2+7=6
333
6+8=6
3+7=6+7=13
2+4

3+7=6
3+7=6
3+7=6
2+7=10
6+=6+=
3
++46+7=10
6+8=6+7=6+77=10
13
=+7
2+7=8
3
++7=6
2+8=

+4=6
3
2+7=13
3+4=6
613
6+7=6
2+8=3
3++==13+=
3++7=1
3170.08408093
5++8=+7=13
2+4=6+7=10=6+7=+7=
3+8=6
3+7=1
3+8=10
6+=
3+8=13
6+7=6
3+7=13
2+4=6
=+7=13
2+7=6
3+8=1
3+8=13
++=63
3+8=6++=13
3+8=6
2+8=13
6+7=
3
+7=6
3
2+8=10
6+8=13+8=6
3
2+8=6
3+8=6
3++=13
2+8=6
+
6+7=1
nan

/home/iris/programs/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:79: RuntimeWarning: overflow encountered in multiply
/home/iris/programs/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:90: RuntimeWarning: invalid value encountered in multiply
/home/iris/programs/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:95: RuntimeWarning: invalid value encountered in multiply
/home/iris/programs/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:100: RuntimeWarning: invalid value encountered in multiply
/home/iris/programs/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:126: RuntimeWarning: invalid value encountered in less



5++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
nan
5++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
nan
5++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
nan
5++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
nan
5++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

KeyboardInterrupt: 

In [35]:
- G(t) = sigma(Wgx * X(t) + Wgh * H(t-1) + Bg) # Input gate
- I(t) = sigma(Wix * X(t) + Wih * H(t-1) + Bi) # Input
- F(t) = sigma(Wfx * X(t) + Wfh * H(t-1) + Bf) # Forget gate
- O(t) = sigma(Wox * X(t) + Woh * H(t-1) + Bo) # Output gate
- S(t) = G(t) .* I(t) + S(t-1) .* F(t) 
- H(t) = sigma(S(t)) .* O(t)
- Ak(t) = Wkh * H(t) + Bk
- Yk(t) = softmax(Ak(t))

SyntaxError: invalid syntax (<ipython-input-35-5072dcf1ac5a>, line 5)